In [1]:
import os, torch, cv2
import numpy as np
from tqdm import tqdm
from scipy import ndimage

from utils.dataloader import get_preprocessing
from utils.config import load_wdb_config, load_setting
from utils.network import Litsmp

THRESHOLD = 0.75

/home/u7085556/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/u7085556/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationESs
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
def connectTH(mask, map, mode=1, threshold=150):
    # identify pixel connected size
    pgroup, Nlabels = ndimage.measurements.label(map)
    label_size = [(pgroup == label).sum() for label in range(Nlabels + 1)]
    # remove those above a threshold
    mode ^= 1
    for label,size in enumerate(label_size):
        if size < threshold:
            mask[pgroup == label] = mode

In [3]:
pretrain_path = './result/U+_nc_moreaug_FTL/'
cfgpath = os.path.join(pretrain_path, 'expconfig.yaml')
weight = os.path.join(pretrain_path, 'epoch=75-step=5396.ckpt')
ds_dict = load_setting()

In [ ]:
Public_Image = ds_dict['public_root']
opts_dict = load_wdb_config(cfgpath)
Public_save_path = os.path.join(ds_dict['inference_root'], opts_dict['expname'])
os.makedirs(Public_save_path, exist_ok=True)

preprocess = get_preprocessing()

model = Litsmp.load_from_checkpoint(weight, opts_dict=opts_dict)
model.eval()
height = opts_dict['aug']['resize_height']
width = height * 2
imagePaths = [image_id for image_id in os.listdir(Public_Image)]
for image_id in tqdm(imagePaths):
    image = cv2.imread(os.path.join(Public_Image, image_id))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    origin_h, origin_w, _ = image.shape
    if image.shape != (height, width, 3):
        image = cv2.resize(image, (width, height), interpolation=cv2.INTER_LANCZOS4)
    image = preprocess(image=image)['image']
    image = image.unsqueeze(0)
    with torch.no_grad():
        mask = torch.sigmoid(model(image)).squeeze().cpu().numpy()
    mask = cv2.resize(mask, (origin_w, origin_h), interpolation=cv2.INTER_LANCZOS4)
    mask = np.where(mask > THRESHOLD, 1, 0)
    origin = np.sum(mask == 1)

    connectTH(mask, mask, mode=1, threshold=500)
    connectTH(mask, mask^1, mode=0, threshold=500)
    
    process = np.sum(mask == 1)
    print(origin, process)

In [19]:
# find the smallest connectTH in trainset
imagePaths = ds_dict['dataset_root']
maskPaths = os.path.join(imagePaths, 'Train_Masks')
mincp = 2000

for image_id in os.listdir(maskPaths):
    mask = cv2.imread(os.path.join(maskPaths, image_id), 0)
    pgroup, Nlabels = ndimage.measurements.label(mask)
    for label in range(Nlabels + 1):
        label_size = (pgroup == label).sum()
        if label_size < mincp:
            mincp = label_size
            print(image_id, mincp)

00000644.png 1994
00000218.png 1049
00000220.png 649
00000186.png 444
00000302.png 425
00000306.png 329
00000536.png 98
